# Autoencoder for MNIST compared to supervised model

### Imports and load the MNIST dataset

In [5]:
import tensorflow as tf

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize data
x_train = x_train / 255.0
x_test = x_test / 255.0


### Vector size for latent variables and number of epochs

In [6]:
LATENT_SIZE = 32
EPOCHS = 10

### Define the autoencoder model consisting of an encoder and decoder

In [7]:
# Autoencoder using CNN layers
encoder = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(28, 28)),
    tf.keras.layers.Conv2D(16, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(LATENT_SIZE, activation="relu")
])

decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(7 * 7 * 32, input_shape=(LATENT_SIZE,)),
    tf.keras.layers.Reshape((7, 7, 32)),
    tf.keras.layers.Conv2DTranspose(32, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2DTranspose(16, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2DTranspose(1, (3, 3), activation="sigmoid", padding="same"),
    tf.keras.layers.Reshape((28, 28))
])

img = tf.keras.layers.Input(shape = (28, 28))
latent_vector = encoder(img)
output = decoder(latent_vector)
autoencoder = tf.keras.models.Model(inputs = img, outputs = output)

### Compile and train the autoencoder

In [8]:
autoencoder.compile("nadam", loss = "binary_crossentropy")

autoencoder.fit(x_train, x_train, epochs=EPOCHS)

Epoch 1/10
1875/1875 [==============================] - 34s 17ms/step - loss: 0.1125
Epoch 2/10
1875/1875 [==============================] - 33s 18ms/step - loss: 0.0858
Epoch 3/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0828
Epoch 4/10
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0813
Epoch 5/10
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0803
Epoch 6/10
1875/1875 [==============================] - 31s 17ms/step - loss: 0.0795
Epoch 7/10
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0790
Epoch 8/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0785
Epoch 9/10
1875/1875 [==============================] - 31s 16ms/step - loss: 0.0782
Epoch 10/10
1875/1875 [==============================] - 32s 17ms/step - loss: 0.0779


# Fine-tune the model with supervised learning with 10% of the training data

In [9]:
# Set up the model
model = tf.keras.models.Sequential([
    encoder,
    tf.keras.layers.Dense(10, activation="softmax")
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Use 1% of the labeled training examples for supervised learning
x_train_supervised = x_train[:600]
y_train_supervised = y_train[:600]

# Train the model
model.fit(
    x_train_supervised,
    y_train_supervised,
    epochs=EPOCHS,
)

# Evaluate the model on the test set
accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy after fine-tuning: {accuracy}")

Epoch 1/10
19/19 [==============================] - 1s 8ms/step - loss: 2.6032 - accuracy: 0.1250
Epoch 2/10
19/19 [==============================] - 0s 9ms/step - loss: 2.0054 - accuracy: 0.4117
Epoch 3/10
19/19 [==============================] - 0s 8ms/step - loss: 1.5598 - accuracy: 0.5950
Epoch 4/10
19/19 [==============================] - 0s 7ms/step - loss: 1.0369 - accuracy: 0.7217
Epoch 5/10
19/19 [==============================] - 0s 8ms/step - loss: 0.6895 - accuracy: 0.7917
Epoch 6/10
19/19 [==============================] - 0s 9ms/step - loss: 0.5162 - accuracy: 0.8383
Epoch 7/10
19/19 [==============================] - 0s 8ms/step - loss: 0.4100 - accuracy: 0.8750
Epoch 8/10
19/19 [==============================] - 0s 8ms/step - loss: 0.3342 - accuracy: 0.9017
Epoch 9/10
19/19 [==============================] - 0s 8ms/step - loss: 0.2895 - accuracy: 0.9233
Epoch 10/10
19/19 [==============================] - 0s 8ms/step - loss: 0.2556 - accuracy: 0.9417
Accuracy after fine